In [45]:
import requests
import furl
import pandas as pd
import time
from bs4 import BeautifulSoup

url = furl.furl("https://scholar.google.com/scholar?start=0&q=machine+learning&hl=en&as_sdt=0,20&as_ylo=2022")

def get_all_articles(start: int):
    url.args["start"] = start
    response = requests.get(
        url=url.url
    )
    print(response.text)
    soup = BeautifulSoup(response.content, "lxml")
    return soup.find_all("div", class_="gs_ri")

def get_article_data(article):
    title = article.find("h3", class_="gs_rt").get_text(strip=False)
    publication_info = article.find("div", class_="gs_a").get_text(strip=False)
    citation_bar = article.find("div", class_="gs_fl gs_flb").find_all("a")
    cited_by = next((cite.text for cite in citation_bar if "Cited by" in cite.text), None)
    print(title, publication_info, cited_by)
    return title, publication_info, cited_by

raw_data = []
start = 0
all_articles = get_all_articles(start)
while len(all_articles) > 0:
    raw_data = raw_data + [get_article_data(article) for article in all_articles]
    start += 10
    time.sleep(2.)
    all_articles = get_all_articles(start)
    
data = pd.DataFrame(data=raw_data, columns=["title", "publication_info", "cited_by"])
data.to_csv("ml_articles_info.csv")



<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
<head><meta http-equiv="content-type" content="text/html; charset=utf-8"><meta name="viewport" content="initial-scale=1"><title>https://scholar.google.com/scholar?start=0&amp;q=machine+learning&amp;hl=en&amp;as_sdt=0%2C20&amp;as_ylo=2022</title></head>
<body style="font-family: arial, sans-serif; background-color: #fff; color: #000; padding:20px; font-size:18px; overscroll-behavior:contain;" onload="e=document.getElementById('captcha');if(e){e.focus();} if(solveSimpleChallenge) {solveSimpleChallenge(,);}">
<div style="max-width:400px;">
<hr noshade size="1" style="color:#ccc; background-color:#ccc;"><br>
<form id="captcha-form" action="index" method="post">
<noscript>
<div style="font-size:13px;">
  In order to continue, please enable javascript on your web browser.
</div>
</noscript>
<script src="https://www.google.com/recaptcha/api.js" async defer></script>
<script>var submitCallback = function(response) {document